In [1]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Измените на ERROR, чтобы уменьшить количество логов
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("process_2_days.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Получение параметров подключения к ClickHouse
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Подключение к ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

# Установка текущей даты как 26 ноября 2024 года
current_date = date(2024, 11, 26)

def process_for_2_days():
    global current_date
    for day in range(2):
        logger.info(f"=== Обработка даты {current_date} ===")

        target_date = current_date
        previous_date = current_date - timedelta(days=1)

        # Проверяем, есть ли данные за предыдущую дату
        query_check_prev_date = f"""
        SELECT COUNT(*) FROM {database_name}.OlapCube_VNV WHERE Dates = '{previous_date}'
        """
        prev_date_count = client.execute(query_check_prev_date)[0][0]
        if prev_date_count == 0:
            logger.warning(f"Нет данных за предыдущую дату {previous_date}. Переходим к следующей дате.")
            current_date += timedelta(days=1)
            continue

        # Шаг 1: Извлечение данных
        prev_dict, target_dict = fetch_previous_and_target_data(target_date, previous_date)
        if not prev_dict or not target_dict:
            logger.error("Не удалось извлечь необходимые данные. Переходим к следующей дате.")
            current_date += timedelta(days=1)
            continue

        # Шаг 2: Расчет Status_P
        status_updates = calculate_status_p(prev_dict, target_dict, target_date)
        if status_updates:
            update_status_p(status_updates, target_date)
        else:
            logger.info("Нет записей для обновления Status_P.")

        # Шаг 3: Рассчитать и обновить балансы
        calculate_and_update_balances(target_date)

        # Шаг 4: Корректировка статуса
        adjust_status(target_date)

        # Шаг 5: Обновление счётчиков
        calculate_and_update_counters(target_date, previous_date)

        # Переходим к следующей дате
        current_date += timedelta(days=1)

def fetch_previous_and_target_data(target_date, previous_date):
    """
    Извлекает данные для предыдущей и target_date из OlapCube_VNV.
    Возвращает словари prev_dict и target_dict.
    """
    try:
        logger.info(f"Обработка предыдущей даты: {previous_date}")
        logger.info(f"Обработка target_date: {target_date}")

        # Извлечение данных за предыдущую дату
        query_prev = f"""
        SELECT 
            serialno, Status, sne, ppr, ll, oh, BR, daily_flight_hours, repair_days, RepairTime
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{previous_date}'
        """
        prev_data = client.execute(query_prev)
        logger.info(f"Извлечено {len(prev_data)} записей за предыдущую дату {previous_date}.")

        # Извлечение данных за target_date, включая текущее значение Status_P
        query_target = f"""
        SELECT 
            serialno, Status_P, ll, oh, BR, daily_flight_hours
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        """
        target_data = client.execute(query_target)
        logger.info(f"Извлечено {len(target_data)} записей за target_date {target_date}")

        # Преобразование данных в словари для быстрого доступа
        prev_dict = {row[0]: row for row in prev_data}
        target_dict = {row[0]: row for row in target_data}

        return prev_dict, target_dict
    except Exception as e:
        logger.error(f"Ошибка при извлечении данных: {e}", exc_info=True)
        return None, None

def calculate_status_p(prev_dict, target_dict, target_date):
    """
    Рассчитывает Status_P для target_date на основе предыдущих данных и условий.
    Возвращает список обновлений: (Status_P, serialno, target_date)
    """
    status_updates = []
    unresolved = []
    changed_serialnos = []

    for serialno, target_row in target_dict.items():
        target_status_p = target_row[1]  # Текущее значение Status_P
        target_ll = target_row[2]
        target_oh = target_row[3]
        target_BR = target_row[4]
        target_daily_flight_hours = target_row[5]

        prev_row = prev_dict.get(serialno, None)
        if not prev_row:
            logger.warning(f"Не найдены данные за предыдущую дату для serialno: {serialno}")
            unresolved.append(serialno)
            continue

        prev_status = prev_row[1]
        prev_sne = prev_row[2]
        prev_ppr = prev_row[3]
        prev_ll = prev_row[4]
        prev_oh = prev_row[5]
        prev_BR = prev_row[6]
        prev_daily_flight_hours = prev_row[7]
        prev_repair_days = prev_row[8]
        repair_time = prev_row[9]

        # Проверка на None
        if None in [prev_sne, prev_ppr, target_ll, target_oh, target_BR, target_daily_flight_hours]:
            logger.warning(f"Пропускаем запись с serialno: {serialno} из-за отсутствия данных.")
            unresolved.append(serialno)
            continue

        # Приведение типов
        try:
            prev_sne = float(prev_sne)
            prev_ppr = float(prev_ppr)
            target_ll = float(target_ll)
            target_oh = float(target_oh)
            target_BR = float(target_BR)
            target_daily_flight_hours = float(target_daily_flight_hours)
            prev_repair_days = float(prev_repair_days) if prev_repair_days is not None else 0
            repair_time = float(repair_time) if repair_time is not None else 0
        except ValueError as ve:
            logger.warning(f"Некорректные числовые значения для serialno: {serialno}. Ошибка: {ve}")
            unresolved.append(serialno)
            continue

        # Инициализация Status_P
        status_p = None

        # Логика расчета Status_P
        # 1. Эксплуатация
        if prev_status == "Эксплуатация":
            condition_sne = prev_sne < (target_ll - target_daily_flight_hours)
            condition_ppr = prev_ppr < (target_oh - target_daily_flight_hours)

            if condition_sne and condition_ppr:
                status_p = "Эксплуатация"
            else:
                if prev_sne < target_BR:
                    status_p = "Ремонт"
                else:
                    status_p = "Хранение"

        # 2. Ремонт
        elif prev_status == "Ремонт":
            if prev_repair_days < repair_time:
                status_p = "Ремонт"
            else:
                status_p = "Исправен"

        # 3. Исправен, Неактивно, Хранение
        elif prev_status in ["Исправен", "Неактивно", "Хранение"]:
            status_p = prev_status

        else:
            unresolved.append(serialno)
            logger.warning(f"Неизвестный Status_P для serialno: {serialno} с предыдущим статусом: {prev_status}")
            continue

        if status_p:
            # Приведение к нижнему регистру и удаление пробелов для корректного сравнения
            old_status_p_clean = prev_status.strip().lower() if isinstance(prev_status, str) else prev_status
            new_status_p_clean = status_p.strip().lower() if isinstance(status_p, str) else status_p

            if old_status_p_clean != new_status_p_clean:
                status_updates.append((status_p, serialno, target_date))
                changed_serialnos.append(serialno)
                logger.info(f"serialno: {serialno} - Status_P изменился с '{prev_status}' на '{status_p}'")
            else:
                pass  # Если статус не изменился, логирование не требуется
        else:
            unresolved.append(serialno)
            logger.warning(f"Не удалось определить Status_P для serialno: {serialno}")

    if unresolved:
        logger.error(f"Не удалось определить Status_P для следующих serialno: {', '.join(unresolved)}")

    logger.info(f"Определено {len(status_updates)} записей для обновления Status_P.")
    if changed_serialnos:
        logger.info(f"Изменились статусы для следующих serialno: {', '.join(changed_serialnos)}")
        logger.info(f"Общее количество измененных serialno: {len(changed_serialnos)}")
    else:
        logger.info("Изменения статусов не зафиксированы.")

    return status_updates

def update_status_p(status_updates, target_date):
    """
    Обновляет поле Status_P в таблице OlapCube_VNV для target_date.
    """
    try:
        for status_p, serialno, date in status_updates:
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE Status_P = '{status_p}' 
            WHERE serialno = '{serialno}' AND Dates = '{date}'
            """
            try:
                client.execute(update_query)
                logger.info(f"Updated Status_P for serialno: {serialno}, Dates: {date} to '{status_p}'")
            except Exception as e_inner:
                logger.error(f"Ошибка при обновлении Status_P для serialno: {serialno}, Dates: {date}. Ошибка: {e_inner}", exc_info=True)
                continue  # Продолжаем обработку остальных записей
        logger.info(f"Обновлено {len(status_updates)} записей в OlapCube_VNV для даты {target_date}.")
    except Exception as e:
        logger.error(f"Ошибка при обновлении Status_P: {e}", exc_info=True)

def calculate_and_update_balances(target_date):
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total, 
    stock_mi8t, stock_mi17, stock_total для target_date.
    """
    try:
        # 1. Рассчитать balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]
        logger.info(f"balance_mi8t_count: {balance_mi8t_count}")

        # Получить mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi8t_count_result = client.execute(query_mi8t_count)
        if mi8t_count_result:
            mi8t_count = mi8t_count_result[0][0]
            balance_mi8t = balance_mi8t_count - mi8t_count
            logger.info(f"balance_mi8t: {balance_mi8t} (balance_mi8t_count: {balance_mi8t_count} - mi8t_count: {mi8t_count})")
        else:
            logger.warning(f"mi8t_count не найден для даты {target_date}. Устанавливаем balance_mi8t = balance_mi8t_count.")
            balance_mi8t = balance_mi8t_count

        # 2. Рассчитать balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]
        logger.info(f"balance_mi17_count: {balance_mi17_count}")

        # Получить mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi17_count_result = client.execute(query_mi17_count)
        if mi17_count_result:
            mi17_count = mi17_count_result[0][0]
            balance_mi17 = balance_mi17_count - mi17_count
            logger.info(f"balance_mi17: {balance_mi17} (balance_mi17_count: {balance_mi17_count} - mi17_count: {mi17_count})")
        else:
            logger.warning(f"mi17_count не найден для даты {target_date}. Устанавливаем balance_mi17 = balance_mi17_count.")
            balance_mi17 = balance_mi17_count

        # 3. Рассчитать balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")

        # 4. Рассчитать stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # 5. Рассчитать stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # 6. Рассчитать stock_empty
        query_stock_empty = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE (ac_typ = '' OR ac_typ IS NULL) AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_empty = client.execute(query_stock_empty)[0][0]
        logger.info(f"stock_empty: {stock_empty}")

        # 7. Рассчитать stock_total
        stock_total = stock_mi8t + stock_mi17 + stock_empty
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17} + stock_empty: {stock_empty})")

        # 8. Обновить таблицу OlapCube_VNV
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{target_date}'
        """
        client.execute(update_query)
        logger.info(f"Баланс и запасы успешно обновлены для даты {target_date}.")

    except Exception as e:
        logger.error(f"Ошибка при расчете или обновлении балансов: {e}", exc_info=True)

def adjust_status(target_date):
    """
    Корректирует поле Status в таблице OlapCube_VNV на основе значения balance_total и статусов в Status_P.
    """
    try:
        # Получить значение balance_total на target_date
        query_balance_total = f"""
        SELECT balance_total
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        balance_total_result = client.execute(query_balance_total)
        if balance_total_result:
            balance_total = balance_total_result[0][0]
            logger.info(f"balance_total на target_date: {balance_total}")
        else:
            logger.error(f"balance_total не найден для даты {target_date}. Завершаем выполнение.")
            return

        # Извлечь записи Status_P на target_date
        query_status_p = f"""
        SELECT serialno, Status_P
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        ORDER BY serialno
        """
        status_p_data = client.execute(query_status_p)
        logger.info(f"Извлечено {len(status_p_data)} записей на target_date.")

        # Разделение записей по статусам
        exploitation = [row for row in status_p_data if row[1] == "Эксплуатация"]
        repaired = [row for row in status_p_data if row[1] == "Исправен"]
        inactive = [row for row in status_p_data if row[1] == "Неактивно"]

        logger.info(f"Начальное количество 'Эксплуатация': {len(exploitation)}")
        logger.info(f"Начальное количество 'Исправен': {len(repaired)}")
        logger.info(f"Начальное количество 'Неактивно': {len(inactive)}")

        changed_serialnos = []
        if balance_total == 0:
            # Копируем все значения Status_P в Status
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE Status = Status_P
            WHERE Dates = '{target_date}'
            """
            client.execute(update_query)
            logger.info("Статусы скопированы без изменений.")
        elif balance_total > 0:
            # Меняем первых balance_total записей "Эксплуатация" на "Исправен"
            rows_to_update = exploitation[:int(balance_total)]
            for row in rows_to_update:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE Status = 'Исправен'
                WHERE serialno = '{row[0]}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(row[0])
            logger.info(f"Изменено {len(changed_serialnos)} записей: 'Эксплуатация' -> 'Исправен'. Serialno: {', '.join(changed_serialnos)}")
        elif balance_total < 0:
            # Меняем abs(balance_total) записей "Исправен" или "Неактивно" на "Эксплуатация"
            deficit_count = abs(int(balance_total))
            rows_to_add = repaired[:deficit_count]
            deficit_count -= len(rows_to_add)
            if deficit_count > 0:
                rows_to_add.extend(inactive[:deficit_count])
            for row in rows_to_add:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE Status = 'Эксплуатация'
                WHERE serialno = '{row[0]}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(row[0])
            logger.info(f"Изменено {len(changed_serialnos)} записей: 'Исправен' или 'Неактивно' -> 'Эксплуатация'. Serialno: {', '.join(changed_serialnos)}")

        # Копирование оставшихся значений из Status_P в Status
        update_remaining_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV
        UPDATE Status = Status_P
        WHERE Dates = '{target_date}' AND (Status IS NULL OR Status = '')
        """
        client.execute(update_remaining_query)
        logger.info("Оставшиеся значения скопированы из Status_P в Status.")

    except Exception as e:
        logger.error(f"Ошибка при корректировке статусов: {e}", exc_info=True)

def calculate_and_update_counters(target_date, previous_date):
    """
    Рассчитывает и обновляет счётчики sne, ppr и repair_days в таблице OlapCube_VNV на target_date.
    Логика обновления зависит от текущего значения поля Status и предыдущего Status_P.
    """
    try:
        logger.info(f"Previous date: {previous_date}")

        # Получить все записи на target_date
        query_target_records = f"""
        SELECT 
            serialno, 
            Status, 
            Status_P, 
            daily_flight_hours,
            sne,
            ppr,
            repair_days
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        """
        target_records = client.execute(query_target_records)
        logger.info(f"Извлечено {len(target_records)} записей на target_date.")

        # Получить все записи на previous_date
        query_previous_records = f"""
        SELECT 
            serialno, 
            Status_P,
            sne,
            ppr,
            repair_days
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{previous_date}'
        """
        previous_records = client.execute(query_previous_records)
        previous_records_dict = {row[0]: row[1:] for row in previous_records}
        logger.info(f"Извлечено {len(previous_records)} записей на предыдущую дату.")

        # Разделение записей по статусам
        records_by_status = {
            "Эксплуатация": [],
            "Исправен": [],
            "Ремонт": [],
            "Хранение": [],
            "Неактивно": []
        }

        unknown_statuses = set()

        for record in target_records:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record
            if status in records_by_status:
                records_by_status[status].append(record)
            else:
                unknown_statuses.add(status)

        if unknown_statuses:
            logger.warning(f"Найдены неизвестные статусы: {', '.join(unknown_statuses)}. Эти записи будут пропущены.")

        changed_serialnos = []

        # Обработка статуса "Эксплуатация"
        for record in records_by_status["Эксплуатация"]:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

            # Получить предыдущие значения
            if serialno in previous_records_dict:
                status_p_prev_val, sne_prev_val, ppr_prev_val, repair_days_prev_val = previous_records_dict[serialno]
            else:
                logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                sne_prev_val = 0.0
                ppr_prev_val = 0.0
                repair_days_prev_val = None

            # Вычислить новые значения
            sne_new = sne_prev_val + daily_flight_hours
            ppr_new = ppr_prev_val + daily_flight_hours
            repair_days_new = repair_days_prev_val  # Не изменяется

            # Проверить необходимость обновления
            if sne_new != sne_prev or ppr_new != ppr_prev:
                # Форматирование значений для SQL
                sne_new_sql = f"{sne_new}"
                ppr_new_sql = f"{ppr_new}"
                repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                # Создание UPDATE запроса
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE 
                    sne = {sne_new_sql},
                    ppr = {ppr_new_sql},
                    repair_days = {repair_days_new_sql}
                WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено {len(records_by_status['Эксплуатация'])} записей статуса 'Эксплуатация'.")

        # Обработка статуса "Исправен"
        for record in records_by_status["Исправен"]:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

            # Получить предыдущие значения
            if serialno in previous_records_dict:
                status_p_prev_val, sne_prev_val, ppr_prev_val, repair_days_prev_val = previous_records_dict[serialno]
            else:
                logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                sne_prev_val = 0.0
                ppr_prev_val = 0.0
                repair_days_prev_val = None

            # Вычислить новые значения
            sne_new = sne_prev_val  # Не изменяется

            if status_p_prev_val == "Ремонт":
                ppr_new = 0.0
                repair_days_new = None
            else:
                ppr_new = ppr_prev_val
                repair_days_new = repair_days_prev_val  # Не изменяется

            # Проверить необходимость обновления
            needs_update = False
            if ppr_new != ppr_prev:
                needs_update = True
            if repair_days_new != repair_days_prev:
                needs_update = True

            if needs_update:
                # Форматирование значений для SQL
                sne_new_sql = f"{sne_new}"
                ppr_new_sql = f"{ppr_new}"
                repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                # Создание UPDATE запроса
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE 
                    sne = {sne_new_sql},
                    ppr = {ppr_new_sql},
                    repair_days = {repair_days_new_sql}
                WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено {len(records_by_status['Исправен'])} записей статуса 'Исправен'.")

        # Обработка статуса "Ремонт"
        for record in records_by_status["Ремонт"]:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

            # Получить предыдущие значения
            if serialno in previous_records_dict:
                status_p_prev_val, sne_prev_val, ppr_prev_val, repair_days_prev_val = previous_records_dict[serialno]
            else:
                logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                sne_prev_val = 0.0
                ppr_prev_val = 0.0
                repair_days_prev_val = None

            # Вычислить новые значения
            sne_new = sne_prev_val  # Не изменяется
            ppr_new = ppr_prev_val  # Не изменяется

            if status_p_prev_val == "Эксплуатация":
                repair_days_new = 1.0
            elif repair_days_prev_val is not None:
                repair_days_new = repair_days_prev_val + 1.0
            else:
                repair_days_new = 1.0

            # Проверить необходимость обновления
            if repair_days_new != repair_days_prev:
                # Форматирование значений для SQL
                sne_new_sql = f"{sne_new}"
                ppr_new_sql = f"{ppr_new}"
                repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                # Создание UPDATE запроса
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE 
                    sne = {sne_new_sql},
                    ppr = {ppr_new_sql},
                    repair_days = {repair_days_new_sql}
                WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено {len(records_by_status['Ремонт'])} записей статуса 'Ремонт'.")

        # Обработка статуса "Хранение" и "Неактивно"
        for status in ["Хранение", "Неактивно"]:
            for record in records_by_status[status]:
                serialno, status_current, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

                # Получить предыдущие значения
                if serialno in previous_records_dict:
                    _, sne_prev_val, ppr_prev_val, _ = previous_records_dict[serialno]
                else:
                    logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                    sne_prev_val = 0.0
                    ppr_prev_val = 0.0

                # Вычислить новые значения (перенос с предыдущей даты)
                sne_new = sne_prev_val
                ppr_new = ppr_prev_val
                repair_days_new = repair_days_prev  # Не изменяется

                # Проверить необходимость обновления
                needs_update = False
                if sne_new != sne_prev:
                    needs_update = True
                if ppr_new != ppr_prev:
                    needs_update = True
                if repair_days_new != repair_days_prev:
                    needs_update = True

                if needs_update:
                    # Форматирование значений для SQL
                    sne_new_sql = f"{sne_new}"
                    ppr_new_sql = f"{ppr_new}"
                    repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                    # Создание UPDATE запроса
                    update_query = f"""
                    ALTER TABLE {database_name}.OlapCube_VNV 
                    UPDATE 
                        sne = {sne_new_sql},
                        ppr = {ppr_new_sql},
                        repair_days = {repair_days_new_sql}
                    WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                    """
                    client.execute(update_query)
                    changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено записей статусов 'Хранение' и 'Неактивно'.")

        logger.info(f"Обновлено всего {len(changed_serialnos)} записей: sne, ppr и repair_days.")
        if changed_serialnos:
            logger.info(f"Serialnos: {', '.join(changed_serialnos)}")

    except Exception as e:
        logger.error(f"Ошибка при обновлении счётчиков: {e}", exc_info=True)

if __name__ == "__main__":
    process_for_2_days()


2024-12-01 19:01:56,260 - INFO - === Обработка даты 2024-11-26 ===
2024-12-01 19:01:56,271 - INFO - Обработка предыдущей даты: 2024-11-25
2024-12-01 19:01:56,272 - INFO - Обработка target_date: 2024-11-26
2024-12-01 19:01:56,380 - INFO - Извлечено 420 записей за предыдущую дату 2024-11-25.
2024-12-01 19:01:56,456 - INFO - Извлечено 420 записей за target_date 2024-11-26
2024-12-01 19:01:56,457 - INFO - serialno: 2228С78 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-01 19:01:56,458 - INFO - serialno: 1228С101 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-01 19:01:56,461 - INFO - serialno: С5381040 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-01 19:01:56,462 - INFO - serialno: С6384678 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-01 19:01:56,463 - INFO - serialno: СВ78В615 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-01 19:01:56,466 - INFO - Определено 5 записей для обновления Status_P.
2024-12-01 19:01:56,467 - INFO - Изменились статусы для след